In [1]:
import cv2
import numpy as np
from scipy.signal import convolve2d
from tqdm import tqdm

In [2]:
#Configuration

K = 5 #Number of mixture models
T = 0.2 #Min portion of data accounted for Background
alpha=0.5 #pi update param
eps = 1e-5
low_prior_weight = 0.01
inf = 1e9
high_variance=1
in_video_file_path = 'Data/umcp.mpg' #path to input audio
out_path = '' #directory for foreground and background videos
frame_size = (240,352)

In [3]:
#model params

#For each pixel, K set of mean,var and pi are required; 
pi = np.ones((*frame_size,K))/K # pi-> (MxNxK)
mean = np.zeros((*frame_size,K))
var = np.ones((*frame_size,K))/K/K #Covariance matrix

params = {
    'pi':pi,
    'mean':mean,
    'var':var,
    'sorted_gaussians_idx': None,
    'B' : None
}

In [4]:
# utils
import warnings
np.seterr(all='warn')
warnings.filterwarnings('error')
def KNN(x):
    win_size = 5
    win = np.ones((win_size,win_size))/(win_size*win_size)
    out = convolve2d(x.squeeze(),win,mode='same')
    out = (out>0.5)*1
    return out[:,:,np.newaxis]

def gpdf_gray(x,params):
    #gaussian probability density function
    mean = params['mean']
    var = params['var']
    mahalD = (x-mean)*(x-mean)/(var+eps)
    mdn = mahalD>100
    mahalD = mahalD*(1-mdn)+ mdn*100           
    return 1/(np.sqrt(2*np.pi*(var+eps))) * np.exp(-mahalD/2)

def update_params(x,params,M,alpha=alpha):
    #M is a bool matrix(true and false)
    rho = alpha*gpdf_gray(x,params)
    for r in range(M.shape[0]):
        for c in range(M.shape[1]):
            no_match = True
            for k in range(M.shape[2]):
                if M[r][c][k]:
                    no_match = False
                    break
            if no_match:
                # replace the least probable distribution
                min_dist_idx = np.argmin(params['pi'][r][c])
                params['pi'][r][c][min_dist_idx] = low_prior_weight
                params['mean'][r][c][min_dist_idx] = x[r][c]
                params['var'][r][c][min_dist_idx] = high_variance
            else:
                for k in range(M.shape[2]):
                    params['pi'][r][c][k] = (1-alpha)*params['pi'][r][c][k] + alpha*M[r][c][k]
                    
                    if M[r][c][k] == True:
                        if rho[r][c][k] > 1:
                            rho[r][c][k] = 0.99
                        params['mean'][r][c][k] = (1-rho[r][c][k])*params['mean'][r][c][k] + \
                                                  rho[r][c][k]*x[r][c]

                        params['var'][r][c][k] = ((1-rho[r][c][k])*params['var'][r][c][k] \
                                              + rho[r][c][k]*((x[r][c]-params['mean'][r][c][k])**2))
                
                
            if params['var'][r][c][k] < 0:
                params['var'][r][c][k] *= (-1)
            # renormalise
    params['pi'] = params['pi']/np.sum(params['pi'],axis=2)[:,:,np.newaxis]
                    
            

def matching(x,params):
    #if x-mean < 2.5*var then its a match
    mean = params['mean']
    var = params['var']
    return np.abs(x-mean) < 2.5*var

def background_model(x,params):
    pi_by_var = -1*params['pi']/np.sqrt(params['var'] + eps)
    params['sorted_gaussians_idx'] = np.argsort(pi_by_var,axis=2)
    pi_sorted = -1*np.sort(-1*params['pi'],axis=2)
    M = frame_size[0]
    N = frame_size[1]
    params['B'] = np.zeros_like(x)
    for r in range(M):
        for c in range(N):
            cumsum = 0
            for k in range(K):
                cumsum += pi_sorted[r][c][k]
                params['B'][r][c] = k+1
                if cumsum > T:
                    break
                
            

    matched = matching(x,params)
    background = np.zeros_like(x)
    for r in range(M):
        for c in range(N):
            for k in range(params['B'][r][c][0]):
                if matched[r][c][params['sorted_gaussians_idx'][r][c][k]]:
                    # classify as background
                    background[r][c] = 1
                    break
    
    background = KNN(background)
    return background
                    

In [5]:
#MAIN

def main(outFolder):
    os.mkdir(f'{outFolder}/background')
    os.mkdir(f'{outFolder}/foreground')
    in_video = cv2.VideoCapture(in_video_file_path)
    idx=0
    with tqdm(total=1000) as pbar:
        while (in_video.isOpened()):
            rval, frame = in_video.read()
            idx +=1 
            print(idx)
            if rval:
                #process frame
                frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)[:,:,2] #Take out I from HSI channel
                frame_gray = frame_gray[:,:,np.newaxis] #Append new axis for shape matching
                M = matching(frame_gray,params) #See if pixels match the model
                update_params(frame_gray,params,M) #update the parameters of the model based on matched value
                fg = 1 - background_model(frame_gray,params) #Get foreground mask by subtracting the background mask 

                #Back conversion into of I values to HSI
                frame_fg_gray = (fg.astype('uint8')*frame_gray).squeeze()
                frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
                frame_fg_hsv =np.concatenate((frame_hsv[:,:,:2],frame_fg_gray[:,:,np.newaxis]),axis=2)

                #Back Conversion to color
                frame_fg_color = cv2.cvtColor(frame_fg_hsv,cv2.COLOR_HSV2BGR)
                frame_bg_color = frame - frame_fg_color

                #writing images
                cv2.imwrite(f'{outFolder}/background/background_{idx:03d}.png',frame_bg_color)
                cv2.imwrite(f'{outFolder}/foreground/foreground_{idx:03d}.png',frame_fg_color)

            else:
                break
            pbar.update(1)
    !ffmpeg -framerate 30 -i $outFolder/background/background_%03d.png -c:v libx264 -profile:v high -crf 20 -pix_fmt yuv420p $outFolder/background.mp4
    !ffmpeg -framerate 30 -i $outFolder/foreground/foreground_%03d.png -c:v libx264 -profile:v high -crf 20 -pix_fmt yuv420p $outFolder/foreground.mp4
            
    shutil.rmtree(f'{outFolder}/background')
    shutil.rmtree(f'{outFolder}/foreground')
    in_video.release()

In [ ]:
import os
import shutil
K = 5
T = 0.3
alpha = 0.8
try:
    shutil.rmtree(f'test_{K}_{T}_{alpha}_5')
except Exception as e:
    pass
os.mkdir(f'test_{K}_{T}_{alpha}_5')
main(f'test_{K}_{T}_{alpha}_5')

  0%|                                                  | 0/1000 [00:00<?, ?it/s]

1


  0%|                                          | 1/1000 [00:01<29:27,  1.77s/it]

2


  0%|                                          | 2/1000 [00:05<43:59,  2.64s/it]

...

21


  2%|▊                                      | 21/1000 [01:20<1:09:39,  4.27s/it]

22


  2%|▊                                      | 22/1000 [01:25<1:13:32,  4.51s/it]

23


In [ ]:
import os
import shutil
K = 5
T = 0.7
alpha = 0.001
os.mkdir(f'test_{K}_{T}_{alpha}_5')
main(f'test_{K}_{T}_{alpha}_5')

In [ ]:
import os
import shutil
K = 5
T = 0.85
alpha = 0.1
os.mkdir(f'test_{K}_{T}_{alpha}_5')
main(f'test_{K}_{T}_{alpha}_5')

In [ ]:
import os
import shutil
K = 5
T = 0.08
alpha = 0.1
os.mkdir(f'test_{K}_{T}_{alpha}_5')
main(f'test_{K}_{T}_{alpha}_5')

In [ ]:
import os
import shutil
K = 5
T = 0.08
alpha = 0.01
os.mkdir(f'test_{K}_{T}_{alpha}_5')
main(f'test_{K}_{T}_{alpha}_5')

In [ ]:
import os
import shutil
K = 7
T = 0.7
alpha = 0.3
os.mkdir(f'test_{K}_{T}_{alpha}_5')
main(f'test_{K}_{T}_{alpha}_5')

In [ ]:
import os
import shutil
K = 7
T = 0.4
alpha = 0.01
os.mkdir(f'test_{K}_{T}_{alpha}_5')
main(f'test_{K}_{T}_{alpha}_5')

In [ ]:
import os
import shutil
K = 5
T = 0.8
alpha = 0.1
os.mkdir(f'test_{K}_{T}_{alpha}')
main(f'test_{K}_{T}_{alpha}')

In [ ]:
K = 5
T = 0.5
alpha = 0.01
os.mkdir(f'test_{K}_{T}_{alpha}')
main(f'test_{K}_{T}_{alpha}')

In [ ]:
K = 5
T = 0.3
alpha = 0.1
os.mkdir(f'test_{K}_{T}_{alpha}')
main(f'test_{K}_{T}_{alpha}')

In [ ]:
K = 5
T = 0.3
alpha = 0.01
os.mkdir(f'test_{K}_{T}_{alpha}')
main(f'test_{K}_{T}_{alpha}')

In [ ]:
K = 4
T = 0.4
alpha = 0.01
os.mkdir(f'test_{K}_{T}_{alpha}')
main(f'test_{K}_{T}_{alpha}')

In [ ]:
K = 5
T = 0.2
alpha = 0.05
os.mkdir(f'test_{K}_{T}_{alpha}')
main(f'test_{K}_{T}_{alpha}')